In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from twnews.wtmf.text_processors import Lemmatizer
from twnews.dataset.storage import TweetsStorage, NewsStorage

news_storage = news_storage = NewsStorage()
texts = news_storage.get_texts()

#constants
wm = 1e-2

In [2]:
# lemmatize
lemmatizer = Lemmatizer()
lemmas_list = lemmatizer.split_texts_to_lemmas(texts)
texts = [' '.join(lemma) for lemma in lemmas_list]

2016-03-24 21:28:44.941964: Function split_texts_to_lemmas started with time measure
2016-03-24 21:29:26.586607: Function split_texts_to_lemmas finished in 0m41.6445469856s


In [3]:
# build X and corpus
tvf = TfidfVectorizer()
tfidf_matrix = tvf.fit_transform(texts)

X = tfidf_matrix.transpose()
corpus = tvf.get_feature_names()

In [4]:
import numpy as np
from twnews.timeit import timeit

@timeit
def build_weight_matrix(matrix):
    '''Slow and ugly realization TODO: rewrite'''
    F = X.copy().todense().tolist()    
    W = []
    rows = len(F)
    columns = len(F[0])

    for i in range(rows):
        row = [1 if F[i][j] != 0.0 else wm for j in range(columns)]
        W.append(row)
    return np.array(W)

In [5]:
# build weight matrix
W = build_weight_matrix(X)

2016-03-24 21:29:26.758273: Function build_weight_matrix started with time measure
2016-03-24 21:29:43.587807: Function build_weight_matrix finished in 0m16.829417944s


In [7]:
texts_num = len(texts)
words_num = len(corpus)
dim = 10
print texts_num, words_num
import pickle
with open('/tmp/unused', 'wb') as f:
    pickle.dump(X,f)

7084 14416


In [ ]:
#init_model
P = np.random.rand(dim, words_num)
Q = np.random.rand(dim, texts_num)
P = P*0.2-0.1
Q = Q*0.2-0.1

x = np.array(X.copy().todense().tolist())

In [ ]:
def iteration(P,Q,W,l=20):
    from time import time    
    from scipy import sparse
    
    new_P = P.copy()
    new_Q = Q.copy()
    
    lI = np.identity(dim)*l
    
    start = time()
    
#     for i in range(20):
#         print 'start build P'
#         lW_i = list(W[i])
#         W_i = sparse.spdiags([lW_i], [0], len(lW_i), len(lW_i))
                
#         A = np.array(np.linalg.inv(np.dot(np.dot(Q,W_i),Q.T)+lI))
    
    start = time()
    for i in range(1):#P.shape[1]
        #print 'start build P'
        W_i = np.diag(list(W[i]))
        A0 = np.dot(np.dot(Q,W_i),Q.T)
        A = np.array(np.linalg.inv(A0+lI))
        #B = np.dot(np.dot(Q,W_i),x[i,:].T)
        #P_i = np.dot(A,B)
        #new_P[:,i]=P_i
        #print '%dth iteration' % i
    end = time()    
    print 'old %s secs' % (end - start)
    Q_1 = sparse.csr_matrix(Q)
    start = time()
    for i in range(20):#P.shape[1]
        lW_i = list(W[i])
        W_i = sparse.spdiags([lW_i], [0], len(lW_i), len(lW_i))
        A00 = np.dot(Q_1,W_i)
        A00 = np.array(A00)
        print A00.shape
        #print Q_1.T.shape
        A0 = np.dot(A00,Q.T)
        A = np.array(np.linalg.inv(A0+lI))
        
        #B = np.dot(np.dot(Q,W_i),x[i,:].T)
        #P_i = np.dot(A,B)
        #new_P[:,i]=P_i
        #print '%dth iteration' % i
    end = time()    
    print 'new %s secs' % (end - start)
    
#     print 'start build Q'
#     for j in range(Q.shape[1]):
#         W_j = np.diag(list(W[:,j]))    
#         A = np.array(np.linalg.inv(np.dot(np.dot(P,W_j),P.T)+lI))
#         #B = np.dot(np.dot(P,W_j),x[:,j].T)
#         #Q_j = np.dot(A,B)
#         #new_Q[:,j]=Q_j
#         break
    end = time()    
    print 'old %s secs' % (end - start)
#         if j%1000 == 0:
#             print '%dth iteration' % j
        
    return new_P,new_Q

P,Q = iteration(P,Q,W)

old 2.51062297821 secs
()
